<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/New%20version%20of%20Adagrad%20and%20Adadelta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

--2021-03-28 23:25:28--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-28 23:25:28--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [      <=>           ]  33.20M  5.90MB/s    in 15s     

2021-03-28 23:25:44 (2.18 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.manual_seed(1234)
np.random.seed(1234)

In [5]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

## New veresion of Adagrad

$$\theta_{t+1,i} = \theta_{t,i} - \frac{\eta * k * {G_{t,i}}^2}{\sqrt{G_{t,i}}+\epsilon} \cdot \nabla_\theta J(\theta_{t,i})$$

where:

$$G_{t,i} = G_{t-1,i} + \Big(\nabla_\theta J(\theta_{t,i})\Big)^2$$

In [16]:
class my_Adagrad:
    def __init__(self, model_params, lr=1e-2, init_acc_sqr_grad=0, eps=1e-10, constant=0.9):
        self.model_params = list(model_params)
        self.lr = lr
        self.constant = constant
        self.acc_sqr_grads = [torch.full_like(p, init_acc_sqr_grad) for p in self.model_params]
        self.eps = eps

    def zero_grad(self):
        for param in self.model_params:
            param.grad = None

    @torch.no_grad()
    def step(self):
        for param, acc_sqr_grad in zip(self.model_params, self.acc_sqr_grads):
            acc_sqr_grad.add_(param.grad * param.grad)
            std = acc_sqr_grad.sqrt().add(self.eps)
            param.sub_(self.constant * acc_sqr_grad * acc_sqr_grad * (self.lr / std) * param.grad)

## Adadelta

$$G_{t,i} = E[g^2]_{t,i} = G_{t-1,i} + \Big(\nabla_\theta J(\theta_{t,i})\Big)^2$$

$$g_{t,i} = \nabla_\theta J(\theta_{t,i})$$

$$E[g^2]_{t,i} = \rho E[g^2]_{t-1,i} + (1-\rho)g^2_{t,i}$$

$$E[\Delta \theta^2]_{t-1,i} = \rho E[\Delta \theta^2]_{t-2,i} + (1-\rho)\Delta \theta^2_{t-1,i}$$

$$\theta_{t+1,i} = \theta_{t,i} - \eta \cdot k \cdot {(E[g^2]_{t,i})}^2 \cdot \frac{\sqrt{E[\Delta \theta^2]_{t-1,i} + \epsilon}}{\sqrt{E[g^2]_{t,i}+\epsilon}} \cdot \nabla_\theta J(\theta_{t,i})$$

In [17]:
class my_Adadelta:
    def __init__(self, model_params, lr=1.0, rho=0.9, eps=1e-6, constant=0.9):
        self.model_params = list(model_params)
        self.lr = lr
        self.rho = rho
        self.eps = eps
        self.constant = constant
        self.avg_sqr_grads = [torch.zeros_like(p) for p in self.model_params]
        self.avg_sqr_deltas = [torch.zeros_like(p) for p in self.model_params] 

    def zero_grad(self):
        for param in self.model_params:
            param.grad = None

    @torch.no_grad()
    def step(self):
        for param, avg_sqr_grad, avg_sqr_delta in zip(self.model_params, \
                                                      self.avg_sqr_grads, \
                                                      self.avg_sqr_deltas):
            avg_sqr_grad.mul_(self.rho).add_(param.grad * param.grad * (1 - self.rho))
            std = avg_sqr_grad.add(self.eps).sqrt()
            delta = avg_sqr_delta.add(self.eps).sqrt().div(std).mul(param.grad)
            param.sub_(self.lr * delta * avg_sqr_grad * avg_sqr_grad * self.constant)
            avg_sqr_delta.mul_(self.rho).add_(delta * delta * (1 - self.rho))

In [18]:
# ************* modify this section for later use *************
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # modify this section for later use 
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid12  = torch.nn.Sigmoid()

        self.layer_activations = dict()

    def forward(self, x):

        # modify this section for later use 
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid12(x)
        pred = self.linear_2(x)
        return pred
# ************* modify this section for later use *************

In [19]:
def get_activation(model, layer_name):    
    def hook(module, input, output):
        model.layer_activations[layer_name] = output
    return hook

In [20]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [21]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)


    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [22]:
# ************* modify this section for later use *************
def avg_std_calculator(_hidden_layer_each_neuron_12):

    avg_selectivity12, std_selectivity12 = selectivity(_hidden_layer_each_neuron_12)

    final_selectivity_avg = (avg_selectivity12) / 1
    final_selecvitity_std = (std_selectivity12) / 1

    return final_selectivity_avg, final_selecvitity_std
# ************* modify this section for later use *************

In [23]:
def model_factory(optimizer_name):
    '''
    optimizer_name : choose one of Adagrad, Adadelta, SGD, and Adam 

    '''
    my_model = Model()
    print("my_model:", my_model)
    my_model.to(device)

    # ************* modify this section for later use *************
    my_model.sigmoid12.register_forward_hook(get_activation(my_model, 's12'))
    # ************* modify this section for later use *************

    if optimizer_name == 'Adadelta':
        my_optimizer = torch.optim.Adadelta(my_model.parameters(), lr=1.0)

    # my adadelta 
    elif optimizer_name == 'my_Adadelta':
        my_optimizer = my_Adadelta(my_model.parameters(), lr=1.0, constant=0.9)

    elif optimizer_name == 'Adagrad':
        my_optimizer = torch.optim.Adagrad(my_model.parameters(), lr=0.1)

    # my adagrad
    elif optimizer_name == 'my_Adagrad':
        my_optimizer = my_Adagrad(my_model.parameters(), lr=0.1, constant=0.9)

    else:
        print("ERROR")
    
    print("my_optimizer:", my_optimizer)
    test_acc, sparsity, selectivity_list_avg, selectivity_list_std = selectivity_trainer(optimizer=my_optimizer, model=my_model)
    # ************* modify this section for later use *************
    file_saver = open(f"sparsity_selectivity_4_optim_{optimizer_name}.txt", "w")
    # ************* modify this section for later use *************
    file_saver.write(str(test_acc)+'\n'+str(sparsity)+'\n'+str(selectivity_list_avg)+'\n'+str(selectivity_list_std)+'\n\n')
    file_saver.close()

    # ************* modify this section for later use *************
    if optimizer_name == 'Adadelta':
        !cp sparsity_selectivity_4_optim_Adadelta.txt /content/drive/MyDrive

    elif optimizer_name == 'my_Adadelta':
        # !cp sparsity_selectivity_4_optim_my_Adadelta.txt /content/drive/MyDrive
        # print("Done!")
        pass
    
    elif optimizer_name == 'Adagrad':
        !cp sparsity_selectivity_4_optim_Adagrad.txt /content/drive/MyDrive

    elif optimizer_name == 'my_Adagrad':
        # !cp sparsity_selectivity_4_optim_my_Adagrad.txt /content/drive/MyDrive
        # print("Done!")
        pass
    # ************* modify this section for later use *************

    else:
        print("ERROR")

In [24]:
no_epochs = 1
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    best_test_loss = 1

    selectivity_avg_list = list()
    selectivity_std_list = list()

    # ************* modify this section for later use *************
    final_spareness_12 = list()
    # ************* modify this section for later use *************

    for epoch in range(no_epochs):

        # ************* modify this section for later use *************
        hidden_layer_each_neuron_12 = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        hidden_layer_each_neuron_12 = np.array(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************


        total_train_loss = 0
        total_test_loss = 0

        # ************* modify this section for later use *************
        hidden_layer_activation_list_12 = list()
        # ************* modify this section for later use *************

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            # ***************** sparsity calculation ***************** #
            hidden_layer_activation_list_12.append(model.layer_activations['s12'])

            # ************* modify this section for later use *************
            for activation, label in zip(model.layer_activations['s12'], labels):
                label = label.item()
                with torch.no_grad():
                    activation = activation.numpy()
                for i in range(256):    
                    hidden_layer_each_neuron_12[i][label].append(activation[i])
    
        selectivity_avg, selecvitity_std = avg_std_calculator(hidden_layer_each_neuron_12)
        # ************* modify this section for later use *************
        
        selectivity_avg_list.append(selectivity_avg)
        selectivity_std_list.append(selecvitity_std)

        # this conains activations for all epochs 
        final_spareness_12.append(hidden_layer_activation_list_12)
        # ***************** sparsity calculation ***************** #

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    # ***************** sparsity calculation ***************** #
    sparsity_list12 = sparsity_calculator(final_spareness_12)

    print(sparsity_list12)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list12[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)
    print("selectivity_avg_list", selectivity_avg_list)
    print("selectivity_std_list", selectivity_std_list)

    return test_acc, average_sparsity, selectivity_avg_list, selectivity_std_list

# Adadelta

In [ ]:
model_factory('Adadelta')

# My_Adadelta

In [25]:
model_factory('my_Adadelta')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: <__main__.my_Adadelta object at 0x7fa1e1630850>

Epoch: 1/1, Train Loss: 2.33109222, Test Loss: 2.33121340, Test Accuracy: 0.09740000
[tensor(0.0048, grad_fn=<MeanBackward0>)]
average_sparsity: [0.004784203600138426]
selectivity_avg_list [0.031008223400742604]
selectivity_std_list [0.011259431627352233]


# AdaGrad

In [ ]:
model_factory('Adagrad')

# My_Adagrad

In [26]:
model_factory('my_Adagrad')

my_model: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid12): Sigmoid()
)
my_optimizer: <__main__.my_Adagrad object at 0x7fa1e1421710>

Epoch: 1/1, Train Loss: 2.25971568, Test Loss: 2.16393021, Test Accuracy: 0.17950000
[tensor(0.0049, grad_fn=<MeanBackward0>)]
average_sparsity: [0.0049476888962090015]
selectivity_avg_list [0.03016619704923385]
selectivity_std_list [0.012062421794539691]
